# Submit geometry optimization

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
# General imports.
import ipywidgets as ipw
from IPython.display import clear_output

# AiiDA imports.
%load_ext aiida
%aiida
# AiiDAlab imports.
import aiidalab_widgets_base as awb
from aiida import orm, plugins

from surfaces_tools.utils import wfn

# Custom imports.
from surfaces_tools.widgets import cdxml, computational_resources, editors, inputs

In [ ]:
Cp2kGeoOptWorkChain = plugins.WorkflowFactory("nanotech_empa.cp2k.geo_opt")

In [ ]:
# Structure selector.
build_slab = editors.BuildSlab(title="Build slab")
input_details = inputs.InputDetails()

structure_selector = awb.StructureManagerWidget(
    importers=[
        awb.StructureUploadWidget(title="Import from computer"),
        awb.StructureBrowserWidget(title="AiiDA database"),
        awb.SmilesWidget(title="From SMILES"),
        cdxml.CdxmlUpload2GnrWidget(title="CDXML"),
    ],
    editors=[
        awb.BasicStructureEditor(title="Edit structure"),
        build_slab,
        awb.BasicCellEditor(),
        editors.InsertStructureWidget(title="Insert molecule"),
    ],
    storable=True,
    node_class="StructureData",
)
ipw.dlink((structure_selector, "structure"), (build_slab, "molecule"))
ipw.dlink((structure_selector, "structure"), (input_details, "structure"))
ipw.dlink((input_details, "details"), (build_slab, "details"))
display(structure_selector)

# Code.
code_input_widget = awb.ComputationalResourcesWidget(
    description="CP2K code:", default_calc_job_plugin="cp2k"
)
resources = computational_resources.ProcessResourcesWidget()

output = ipw.Output()

In [ ]:
# Protocol.
protocol = ipw.Dropdown(
    value="standard",
    options=[
        ("Standard", "standard"),
        ("Low accuracy", "low_accuracy"),
        ("Debug", "debug"),
    ],
    description="Protocol:",
    style={"description_width": "initial"},
)

In [ ]:
workflow_description = ipw.Text(
    description="Workflow description:",
    placeholder="Provide the description here.",
    style={"description_width": "initial"},
    layout={"width": "70%"},
)

In [ ]:
ipw.dlink((code_input_widget, "value"), (input_details, "selected_code"))


def prepare_geometry_optimization():
    with output:
        clear_output()
    if not structure_selector.structure_node:
        can_submit, msg = False, "Select a structure first."
    elif not code_input_widget.value:
        can_submit, msg = False, "Select CP2K code."
    else:
        can_submit, msg, parameters = input_details.return_final_dictionary()

    if not can_submit:
        with output:
            print(msg)
            return

    builder = Cp2kGeoOptWorkChain.get_builder()
    builder.protocol = orm.Str(protocol.value)
    builder.metadata.label = (
        "CP2K_CellOpt" if input_details.do_cell_opt else "CP2K_GeoOpt"
    )
    builder.metadata.description = workflow_description.value
    code = orm.load_code(code_input_widget.value)
    builder.code = code
    builder.options = {
        "max_wallclock_seconds": resources.walltime_seconds,
        "resources": {
            "num_machines": resources.nodes,
            "num_mpiprocs_per_machine": resources.tasks_per_node,
            "num_cores_per_mpiproc": resources.threads_per_task,
        },
    }

    builder.structure = structure_selector.structure_node

    # Check if a restart wfn is available.
    wave_function = None
    if structure_selector.structure_node.is_stored:
        wave_function = wfn.structure_available_wfn(
            node=structure_selector.structure_node,
            relative_replica_id=None,
            current_hostname=code.computer.hostname,
            return_path=False,
            dft_params=parameters["dft_params"],
        )
    if wave_function is not None:
        print(f"Restarting from wfn in folder: {wave_function.pk}")
        builder.parent_calc_folder = wave_function

    builder.dft_params = orm.Dict(parameters["dft_params"])
    builder.sys_params = orm.Dict(parameters["sys_params"])

    return builder

In [ ]:
btn_submit = awb.SubmitButtonWidget(
    Cp2kGeoOptWorkChain,
    inputs_generator=prepare_geometry_optimization,
    disable_after_submit=False,
    append_output=True,
)

In [ ]:
# Resources estimation.
resources_estimation = computational_resources.ResourcesEstimatorWidget()
resources_estimation.link_to_resources_widget(resources)
ipw.dlink((input_details, "details"), (resources_estimation, "details"))
ipw.dlink((input_details, "uks"), (resources_estimation, "uks"))
_ = ipw.dlink((code_input_widget, "value"), (resources_estimation, "selected_code"))

# Inputs

In [ ]:
display(input_details, protocol)

# Code and resources

In [ ]:
display(code_input_widget, resources, resources_estimation)

# Submit

In [ ]:
display(workflow_description, btn_submit, output)